<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2023TeamD/blob/dev_branch_EDA_Prediction/Code/ReHealthCo_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReHealthCo EDA-Prozess

## Vorbereitung 

###CleanUp - Eventuell gespeicherte Variablen löschen vor Ausführung des Colab books (Ziel Reproduzierbarkeit)

In [ ]:
# Löschen aller Variablen
%reset -f

Installtion der Package ydata_profiling

In [ ]:
!pip install ydata_profiling

## Importieren der notwendigen Libraries: sqlite3, csv, pandas

In [ ]:
import sqlite3
from sqlite3 import Error
import csv
import pandas as pd
from pandas.util import hash_pandas_object
from ydata_profiling import ProfileReport

### Erstellung einer virtuellen Kopie des GitHub Repositorium DMA2023TeamD in Colab

In [ ]:
!git clone https://github.com/Fuenfgeld/DMA2023TeamD.git

In [ ]:
# Wechsel in den Unterordner csv_data des Github-Projekt-Ordners
%cd /content/DMA2023TeamD/csv_data

In [ ]:
# Auflistung aller Ordnerinhalte
!ls

### Verknüpfung G-Dive mit Colab benötigt????????

In [ ]:
# !mkdir -p /content/drive
# from google.colab import drive
# drive.mount('/content/drive')

### Benennung des rhc-DataFrames und Einlesen des entsprechenden anonymisierten csv-Files von Github

In [ ]:
# Pfad-Abkürzung zum Speicherort der anonymisierten csv-Tabelle
csv_path_anonymrhc = "/content/DMA2023TeamD/csv_data"

In [ ]:
# Einlesen der anonymisierten Tabelle als Pandas DataFrame
rhc_df = pd.read_csv(f"{csv_path_anonymrhc}/anonym_rhcpat.csv")

In [ ]:
# Anzeige des DF-Inhalts
rhc_df

In [ ]:
# Anzeige der Kopfspalten des DF
print(rhc_df.columns.tolist())

In [ ]:
# Diagnosen/Patient
rhc_df.groupby(["RHC_PSEUDONYM", 'DESC_CON']).size()

In [ ]:
rhc_df.groupby(["RHC_PSEUDONYM"]).size()

In [ ]:
rhc_df.groupby(["RHC_PSEUDONYM"]).count()

In [ ]:
df1 = rhc_df.groupby(['RHC_PSEUDONYM', 'GENDER_PAT', 'HEALTHCARE_EXPENSES_PAT', 'HEALTHCARE_COVERAGE_PAT', 'DATENBANKART', 'LEBENDIG', 'AGE', 'CODE_CON', 'DESC_CON', 'CONDITIONDAUER', 'TOTALCOST_MED', 'ENCOUNTERCLASS_ENC', 'BEHANDLUNGSDAUER'])['RHC_PSEUDONYM'].count()
df1

In [ ]:
# ydata-profiling auf anonymisierte Tabelle
ProfileReport(rhc_df)

In [ ]:
ProfileReport(df1)

# Version Check

In [ ]:
#python Version
import sys
sys.version_info

In [ ]:
pd.__version__

In [ ]:
csv.__version__

In [ ]:
# Überprüfen aller Variablen
# %pip freeze

# ANPASSEN!! Calculate the Checksum of the Data Set

In [ ]:
rhcpat_df_hashes = hash_pandas_object(rhcpat_df)
rhccon_df_hashes = hash_pandas_object(rhccon_df)
rhcmed_df_hashes = hash_pandas_object(rhcmed_df)
rhcenc_df_hashes = hash_pandas_object(rhcenc_df)
rhcpatidx_df_hashes = hash_pandas_object(rhcpatidx_df)
rhcpat_schluesseltabelle_df_hashes = hash_pandas_object(rhcpat_schluesseltabelle_df)
rhcpat_schluesseltabelleidx_df_hashes = hash_pandas_object(rhcpat_schluesseltabelleidx_df)
rhc_verein1_df_hashes = hash_pandas_object(rhc_verein1_df)
rhc_verein2_df_hashes = hash_pandas_object(rhc_verein2_df)
rhc_vereinall_df_hashes = hash_pandas_object(rhc_vereinall_df)
anonym_rhcpat_df_hashes = hash_pandas_object(anonym_rhcpat_df)

print(rhcpat_df_hashes)
print(rhccon_df_hashes)
print(rhcmed_df_hashes)
print(rhcenc_df_hashes)
print(rhcpatidx_df_hashes)
print(rhcpat_schluesseltabelle_df_hashes)
print(rhcpat_schluesseltabelleidx_df_hashes)
print(rhc_verein1_df_hashes)
print(rhc_verein2_df_hashes)
print(rhc_vereinall_df_hashes)
print(anonym_rhcpat_df_hashes)

In [ ]:
Liste_rhcetcall_hashes = [rhcpat_df_hashes, rhccon_df_hashes, rhcmed_df_hashes, rhcenc_df_hashes, rhcpatidx_df_hashes, rhcpat_schluesseltabelle_df_hashes, rhcpat_schluesseltabelleidx_df_hashes, rhc_verein1_df_hashes, rhc_verein2_df_hashes, rhc_vereinall_df_hashes, anonym_rhcpat_df_hashes]

In [ ]:
print(Liste_rhcetcall_hashes)

## Value should be 
8511479554227127615

-181358878522955579

6124240394299011028

-2843949348663385963

-7008809488748868895

-3225392982667861325

-7008809488748868895

-5123959212951937834

476094269920066095

7757257148626635020

-8218260507818896620

In [ ]:
print(rhcpat_df_hashes.sum())
print(rhccon_df_hashes.sum())
print(rhcmed_df_hashes.sum())
print(rhcenc_df_hashes.sum())
print(rhcpatidx_df_hashes.sum())
print(rhcpat_schluesseltabelle_df_hashes.sum())
print(rhcpat_schluesseltabelleidx_df_hashes.sum())
print(rhc_verein1_df_hashes.sum())
print(rhc_verein2_df_hashes.sum())
print(rhc_vereinall_df_hashes.sum())
print(anonym_rhcpat_df_hashes.sum())